In [18]:
import pandas as pd

In [38]:
# LSTM for international airline passengers problem with regression framing
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=2):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset
dataframe = read_csv('data/airline-passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
print(dataset.shape)
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 2
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX_r = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX_r = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

(144, 1)


In [37]:
trainY.shape

(93,)

In [34]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back), return_sequences=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX_r, trainY, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX_r)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

Epoch 1/100
93/93 - 0s - loss: 0.0240
Epoch 2/100
93/93 - 0s - loss: 0.0100
Epoch 3/100
93/93 - 0s - loss: 0.0080
Epoch 4/100
93/93 - 0s - loss: 0.0069
Epoch 5/100
93/93 - 0s - loss: 0.0059
Epoch 6/100
93/93 - 0s - loss: 0.0051
Epoch 7/100
93/93 - 0s - loss: 0.0044
Epoch 8/100
93/93 - 0s - loss: 0.0037
Epoch 9/100
93/93 - 0s - loss: 0.0033
Epoch 10/100
93/93 - 0s - loss: 0.0029
Epoch 11/100
93/93 - 0s - loss: 0.0027
Epoch 12/100
93/93 - 0s - loss: 0.0026
Epoch 13/100
93/93 - 0s - loss: 0.0025
Epoch 14/100
93/93 - 0s - loss: 0.0025
Epoch 15/100
93/93 - 0s - loss: 0.0024
Epoch 16/100
93/93 - 0s - loss: 0.0024
Epoch 17/100
93/93 - 0s - loss: 0.0024
Epoch 18/100
93/93 - 0s - loss: 0.0024
Epoch 19/100
93/93 - 0s - loss: 0.0024
Epoch 20/100
93/93 - 0s - loss: 0.0023
Epoch 21/100
93/93 - 0s - loss: 0.0023
Epoch 22/100
93/93 - 0s - loss: 0.0023
Epoch 23/100
93/93 - 0s - loss: 0.0024
Epoch 24/100
93/93 - 0s - loss: 0.0023
Epoch 25/100
93/93 - 0s - loss: 0.0023
Epoch 26/100
93/93 - 0s - loss: 0.

ValueError: in user code:

    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1445 run_step 